conexao com bd


In [184]:
!pip install pymongo[srv,tls]
import pymongo
import json
from pprint import pprint
from datetime import datetime
# nome do usuário que foi cadastrado em Database Access
mongo_user = "lorenzo"
# senha que foi cadastrada em Database Access
mongo_pw = "naruto001"
# host que aparece quando você acessa Connect > Drivers
mongo_host = "clusterbd2.ewikb.mongodb.net"
# nome do cluster que você criou
mongo_app = "Clusterbd2"

# estrutura de conexão, não há necessidade de mudar nada por aqui! ;)
conexao = pymongo.MongoClient("mongodb+srv://" + mongo_user + ":" + mongo_pw + "@" + mongo_host + "/?retryWrites=true&w=majority&appName=" + mongo_app)
conexao.list_database_names()


['db_redecirurgias', 'admin', 'local']

In [185]:
bd = conexao["db_vendalivros"]
print(conexao.list_database_names())

['db_redecirurgias', 'admin', 'local']


criacao das collections


In [186]:
cliente = [
    {
        "estado": {
        "uf": "sp",
        "nome": "sao paulo",
        "regiao": "sudeste"
    },
    "cidade": {
        "cod_cidade": 1,
        "nome": "sao paulo",
        "uf": "sp"
    },
    "endereco": {
        "cod_endereco": 1,
        "rua": "avenida paulista",
        "numero": 1000,
        "complemento": "Apt 2",
        "cep": 12345678,
        "cod_cidade": 1,
        "cod_cliente": 1
    },
    "cliente": {
        "cod_cliente": 1,
        "nome": "joaozinho da silva",
        "data_cadastro": datetime(2024, 1, 15),
        "tipo": "F"
    },
    "pessoa_juridica": {
        "cod_cliente": 2,
        "razao_social": "Empresa FODA",
        "cnpj": "12345678000000"
    },
    "pessoa_fisica": {
        "cod_cliente": 1,
        "cpf": "12345678900",
        "genero": "M",
        "data_nascimento": datetime(1997, 5, 20),
        "cod_empresa": 2
    },
    "telefone": [
        {"id_telefone": "1", "ddd": 11, "numero": 987654321, "cod_cliente": 1},
        {"id_telefone": "2", "ddd": 21, "numero": 876543210, "cod_cliente": 2}
    ]
    }
]

In [187]:
bd.cliente.insert_many(cliente)

InsertManyResult([ObjectId('6723f4173c3202e057eff534')], acknowledged=True)

In [188]:
pedido = [{
    "pedido": {
        "num_pedido": 1,
        "data_emissao": datetime(2022, 2, 1),
        "cod_cliente": 1
    },
    "item_pedido": [
        {"num_item_pedido": 1, "num_pedido": 1001, "cod_produto": 1, "quantidade": 1, "valor_unitario": 50.0}

    ],
    "entrega": {
        "num_entrega": 5001,
        "data": datetime(2022, 2, 5),
        "placa": "ABC123",
        "mot_cnh": "12345678910",
        "mot_nome": "carlos oliveira"
    }
}]

In [189]:
bd.pedido.insert_many(pedido)

InsertManyResult([ObjectId('6723f4183c3202e057eff535')], acknowledged=True)

In [190]:
produto_validation_schema = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["cod_produto", "nome", "data_lancamento", "importado", "preco", "prazo_entrega", "cod_categoria"],
        "properties": {
            "cod_produto": {
                "bsonType": "int",
                "description": "codigo do produt"
            },
            "nome": {
                "bsonType": "string",
                "description": "nome do produto "
            },
            "data_lancamento": {
                "bsonType": "date",
                "description": "data de lançamento do produto"
            },
            "importado": {
                "bsonType": "string",
                "enum": ["S", "N"],
                "description": "produto importado (S/N)"
            },
            "preco": {
                "bsonType": "double",
                "minimum": 0,
                "description": "preco do produto "
            },
            "prazo_entrega": {
                "bsonType": "int",
                "minimum": 0,
                "description": "prazo entrega em dias"
            },
            "cod_categoria": {
                "bsonType": "int",
                "description": "codigo da categoria do produto"
            }
        }
    }
}


bd.create_collection("produto", validator=produto_validation_schema)


Collection(Database(MongoClient(host=['clusterbd2-shard-00-02.ewikb.mongodb.net:27017', 'clusterbd2-shard-00-00.ewikb.mongodb.net:27017', 'clusterbd2-shard-00-01.ewikb.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='Clusterbd2', authsource='admin', replicaset='atlas-12g2hg-shard-0', tls=True), 'db_vendalivros'), 'produto')

In [191]:


produto = [{
    "cod_produto": 1,
    "nome": "celular",
    "data_lancamento": datetime(2021, 12, 1),
    "importado": "N",
    "preco": 1500.0,
    "prazo_entrega": 5,
    "cod_categoria": 1,
    "avaliacao": [
        {"id_avaliacao": "1", "nota": 4, "comentario": "muito bom", "cod_produto": 1, "cod_cliente": 1},
        {"id_avaliacao": "2", "nota": 5, "comentario": "nao trava", "cod_produto": 1, "cod_cliente": 2}
    ],
    "categoria": {
        "cod_categoria": 1,
        "nome": "eletronicos",
        "cod_categoria_pai": None
    }
}]

In [192]:
bd.produto.insert_many(produto)

InsertManyResult([ObjectId('6723f4183c3202e057eff536')], acknowledged=True)

consultas


In [202]:

clientes = bd.cliente.aggregate([
    {
        "$lookup": {
            "from": "cidade",
            "localField": "endereco.cod_cidade",
            "foreignField": "cod_cidade",
            "as": "_"
        }
    },
    {
        "$project": {
            "cliente.nome": 1,
            "cidade.nome": 1,
            "_id": 0
        }
    }
])

for cliente in clientes:
    print(cliente)


{'cidade': {'nome': 'sao paulo'}, 'cliente': {'nome': 'joaozinho da silva'}}


In [201]:
categorias = bd.produto.aggregate([
    {
        "$group": {
            "_id": "$categoria.nome",
            "totalProdutos": { "$sum": 1 }
        }
    },
    {
        "$project": {
            "Categoria": "$_id",
            "totalProdutos": 1,
            "_id": 0
        }
    }
])

for categoria in categorias:
    print(categoria)

{'totalProdutos': 1, 'Categoria': 'eletronicos'}


In [200]:
produtos_mais_vendidos = bd.pedido.aggregate([
    { "$unwind": "$item_pedido" },
    {
        "$group": {
            "_id": "$item_pedido.cod_produto",
            "totalVendas": { "$sum": "$item_pedido.quantidade" }
        }
    },
    { "$sort": { "totalVendas": -1 } },
    {
        "$lookup": {
            "from": "produto",
            "localField": "_id",
            "foreignField": "cod_produto",
            "as": "produto"
        }
    },
    { "$unwind": "$produto" },
    {
        "$project": {
            "_id": 0,
            "Produto": "$produto.nome",
            "totalVendas": 1
        }
    }
])

for produto in produtos_mais_vendidos:
    print(produto)


{'totalVendas': 1, 'Produto': 'celular'}


In [198]:
pedidos = bd.pedido.aggregate([
    {
        "$lookup": {
            "from": "cliente",
            "localField": "pedido.cod_cliente",
            "foreignField": "cliente.cod_cliente",
            "as": "detalhes_cliente"
        }
    },
    { "$unwind": "$detalhes_cliente" },
    { "$unwind": "$item_pedido" },
    {
        "$group": {
            "_id": "$pedido.num_pedido",
            "Cliente": { "$first": "$detalhes_cliente.cliente.nome" },
            "valorTotal": {
                "$sum": { "$multiply": ["$item_pedido.quantidade", "$item_pedido.valor_unitario"] }
            }
        }
    },
    { "$sort": { "valorTotal": -1 } },
    {
        "$project": {
            "_id": 0,
            "Cliente": 1,
            "numPedido": "$_id",
            "valorTotal": 1
        }
    }
])

for pedido in pedidos:
    print(pedido)

{'Cliente': 'joaozinho da silva', 'valorTotal': 1500.0, 'numPedido': 1}


In [199]:
top_categorias = bd.pedido.aggregate([
    { "$unwind": "$item_pedido" },
    {
        "$lookup": {
            "from": "produto",
            "localField": "item_pedido.cod_produto",
            "foreignField": "cod_produto",
            "as": "produto"
        }
    },
    { "$unwind": "$produto" },
    {
        "$group": {
            "_id": "$produto.categoria.nome",
            "totalVendas": { "$sum": "$item_pedido.quantidade" }
        }
    },
    { "$sort": { "totalVendas": -1 } },
    { "$limit": 3 },
    {
        "$project": {
            "Categoria": "$_id",
            "_id": 0,
            "totalVendas": 1
        }
    }
])

for categoria in top_categorias:
    print(categoria)


{'totalVendas': 1, 'Categoria': 'eletronicos'}
